# NOTES ABOUT PHILOSOPHER

# Makefile

In [1]:
%%file Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: wnaseeve <wnaseeve@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2023/04/03 10:22:03 by wnaseeve          #+#    #+#              #
#    Updated: 2023/04/03 10:23:39 by wnaseeve         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME		= philo

SRCS    	= philo.c\
			  philo_errors.c\
			  philo_utils.c\
			  philo_init.c\
			  philo_threads.c\
			  philo_free.c\
			  philo_time.c\
			  philo_actions.c\
			  philo_status.c\

OBJS		= ${SRCS:.c=.o}

HEADER		= philo.h

CC			= cc

CFLAGS		= -g3 -Wall -Wextra -Werror

SANITIZER 	= -ggdb -fsanitize=address -fsanitize=thread -fno-omit-frame-pointer -O1

all:	$(NAME)

$(NAME): $(OBJS)
	@ printf "%b" "\t    $(_CYAN)[Cleaning] $(_CYAN)[all]$(_STARLIGHT_GREEN)\n";
	@ echo ""
	@ echo "\t\033[34;5mCompiling philosophers...\033[0m"
	@ echo ""
	@ $(CC) $(CFLAGS) $(OBJS) $(SANITIZER) -pthread -o philo

%.o: %.c $(HEADER)
		${CC} ${CFLAGS} -I. -c $< -o ${<:.c=.o}

valgrind:
	make && clear && valgrind --leak-check=full --track-origins=yes --show-leak-kinds=all --track-fds=yes --suppressions=supp.supp

#valgrind complet : valgrind --leak-check=full --track-origins=yes --show-leak-kinds=all --tool=helgrind --tool=drd ./philo

gdb:
	make && clear && gdb -tui ./philo

norminette:
	norminette *

clean:
	@printf "%b" "\t    $(_PURPLE)[Cleaning] $(_PURPLE)[all]$(_PINK_ORANGE)\n";
	rm -f $(OBJS)

fclean: clean
	@printf "%b" "\t    $(_ORANGE)[Cleaning] $(_ORANGE)[all]$(_RED)\n";
	rm -f $(NAME)

re:		fclean all

.PHONY:		all clean fclean re

# -------------- Syntaxing -------------- #

_NO                    =        \033[0m
_BOLD                =        \033[1m
_BLUE                =        \033[34m
_CYAN                =        \033[36m
_RED                =        \033[31m
_PURPLE             =        \033[35m
_PINK_ORANGE        =        \033[38;5;215m
_GREY                 =        \033[38;5;234m
_STARLIGHT_BLUE     =        \033[38;5;158m
_STARLIGHT_GREEN    =        \033[38;5;157m
_DEEP_BLUE            =        \033[38;5;69m
_YELLOW             =        \033[38;5;226m
_ORANGE                =         \033[38;5;209m\e[1m

# -------------- Syntaxing -------------- #


Overwriting Makefile


# Header

In [1]:
%%file philo.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   philo.h                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: wnaseeve <wnaseeve@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/03 10:21:18 by wnaseeve          #+#    #+#             */
/*   Updated: 2023/04/03 10:21:37 by wnaseeve         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <string.h>
#include <sys/time.h>

typedef struct CommonData
{
	int				nb_philo;
	int				to_die;
	int				to_eat;
	int				to_sleep;
	int				must_eat;
	int				finished_eating;
	int				end; //stop
	int				*forks;
	struct timeval t_start;
	struct s_philo **philos; // pointeur vers la structure t_philo
	pthread_mutex_t *mutex; // fourchette par personne
	pthread_mutex_t status; // mutex write a chaque printf
	pthread_mutex_t dead; // mutex pour quil recupere la donnee end

}	t_data;

typedef struct s_philo
{
	int				n_meals; // nb de repas
	int				i_philo; // index du philo
	struct timeval t_meal;
	pthread_t		tid; // ID de chaque thread
	t_data			*data; // pointeur sur ma strcture t_data

}	t_philo;

/* ********** INIT FUNCTIONS ********** */
int		ft_initialize1(t_data *data, char *av[], int ac);
int		ft_initialize2(t_data *data);
int		*ft_create_forks(int nb_philo);
pthread_mutex_t	*ft_create_mutex(int nb_philo);
void	ft_init_philos(t_data *data, int i);

/* ********** THREADS FUNCTIONS ********** */
int		ft_create_threads(t_data *data);
void	*ft_routine(void *arg);
int		ft_join_threads(t_data *data);
int		ft_one_philo(t_philo *philo);
int		ft_multiple_philos(t_philo *philo);

/* ********** ACTIONS FUNCTIONS ********** */
void	ft_actions(t_philo *philo, int i_philo);
int		ft_get_index(t_philo *philo, int index);
void	ft_eat(t_philo *philo, int *rfork, int *lfork, int index);

/* ********** STATUS FUNCTIONS ********** */
void	ft_writing_death(t_philo *philo);
void	ft_writing_status(t_philo *philo, char flag);
void	ft_death(t_philo *philo);

/* ********** TIME FUNCTIONS ********** */
int	ft_calc_timestamp(struct timeval start);
struct timeval	ft_now(void);
int	ft_calc_time(struct timeval now, struct timeval start);
void	ft_msleep(t_philo *philo, int interval);

/* ********** ERROR FUNCTIONS ********** */
void	msg_error(char *msg);

/* ********** FREE FUNCTIONS ********** */
void	ft_free_philo(t_philo *philo);
void	ft_free_philos(t_data *data);
void	ft_free_data(t_data *data);

/* ********** UTILS FUNCTIONS ********** */
long	ft_atol(const char *nptr);
int		ft_strlen(char *s);
void	print_tab(t_data *data);


Overwriting philo.h


# Notes

Ici, on cherche a effectuer plusieurs taches simultanement.

un thread : suite logiique d'instructions a l'interieur d'un processus qui est automatiquement geree par le noyau du sys d'exploitation. Plusieurs threads s'executent simultanement, en parallele.

Chaque thread d'un processus possede son propre contexte.
Tous les threads font partie du meme processus, ils partagent le meme espace d'adressage virtuel : la même heap, les mêmes bibliothèques partagées et les mêmes descripteurs de fichiers ouverts.

Ces threads forment un ensemble de pairs. N’importe quel thread peut tuer ou attendre la fin de n’importe quel autre thread dans le même processus.

Difference entre process (fork) et thread : 
	- fork est un processus pere fils dont le fils copie le pere
	- thread ont le meme PID alors que fork non
	- toutes les variables cree partagent la meme adresse memoire dans un thread

pthread_join : bloque execution d'un thread en attendant qu'un autre thread se termine.

Au dessus de 1 millions de thread, ca incremente bizarrement 

Une situation de competition se creer : le data race.
En examinant les résultats, on peut voir que le compte est juste si et seulement si le premier thread termine ses itérations avant que le deuxième commence. Dès que leurs exécutions se chevauchent, le résultat est faussé et toujours inférieur au résultat attendu.

peut-on empêcher un thread de lire une valeur quand un autre thread la modifie ? Oui, du coup, on utilise un mutex pour reguler l'acces aux donnees.

pthread_mutex_lock : verrouiller un mutex jusqu'au deverouillage
pthread_mutex_unlock : 
pthread_mutex_destroy : detruit un mutex deverouille

Un mutex c'est un verrou qui permet de reguler l'acces aux donnees et dempecher que les ressources partagees soient utilisees en meme temps.

Attention aux deadlocks : c'est une situation dans laquelle chaque thread attend une ressource détenue par un autre thread.

PROGRAMME:



# TESTER


arg = 4 ou 5 (optionnel)

Ce que je dois tester:
	- la performance : en millisecondes
	- %%timeit
		-r number of runs
		-n number of loops
- tricky case =
	- ./philo 1 800 200 200	The philosopher should not eat and should die!
	- ./philo 5 800 200 200	No one should die!
	- ./philo 5 800 200 200 7	No one should die and the simulation should stop when all the philosophers has eaten at least 7 times each
	- ./philo 4 410 200 200	No one should die!
	- ./philo 4 310 200 100	A philosopher should die!
	- ./philo 5 800 200 150	No one should die!
	- ./philo 3 610 200 80	No one should die!

In [1]:
%%file tester.py

import unittest
import sys

for i in range(1, len(sys.argv)):
    print(sys.argv[i])
print("Let's test the performance of your philosopher")

Overwriting tester.py


In [18]:
%%timeit -r 10 -n 100
%%bash
./philo 5 200 200 200 3 > /dev/null

# python3 tester.py 6 4 200 200 400

3.09 ms ± 95.7 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
